<a href="https://colab.research.google.com/github/tanishqbololu/PyTorch/blob/main/14_pytorch_rnn_based_qa_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')

df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
# tokenize
def tokenize(text):
    text = text.lower()
    text = text.replace('?','')
    text = text.replace("'","")
    return text.split()

In [ ]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [ ]:
# vocab
vocab = {'<UNK>':0}


In [ ]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question +  tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)



In [ ]:
df.apply(build_vocab, axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [ ]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [ ]:
len(vocab)

324

In [ ]:
# convert word to numerical
def text_to_indices(text, vocab):

    indexed_text = []

    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

In [ ]:
text_to_indices("what is bournvita", vocab)

[1, 2, 0]

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

In [ ]:
class QADataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

        return torch.tensor(numerical_question) , torch.tensor(numerical_answer)

In [ ]:
dataset = QADataset(df, vocab)

In [ ]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
for question, answer in dataloader:
    print(question, answer)

tensor([[ 42, 255,   2, 256,  83, 257, 258]]) tensor([[259]])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([[149]])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[ 42, 137,   2,  62,  39,   3, 322, 323]]) tensor([[6]])
tensor([[ 10,  75, 208]]) tensor([[209]])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([[128]])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([[179]])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([[207]])
tensor([[10, 96,  3, 97]]) tensor([[98]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([[246]])
tensor([[ 10,  96,   3, 104, 239]]) tensor([[240]])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([[65]])
tensor([[  1,   2,   3,  92, 137,  19,   3,  45]]) tensor([[185]])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([[116]])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([[220]])
tensor([[  1,   2,   3, 1

![image.png](attachment:image.png)

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim=50) # turn every word into 50dim vector
        self.rnn = nn.RNN(50, 64, batch_first=True)  # gives both hidden states and final output, i.e we arenot usind Sq container (it needs only one o/p per layer)
        self.fc = nn.Linear(64, vocab_size)

    def forward(self, question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))
        return output

In [ ]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [ ]:
learning_rate = 0.001
epochs = 20

In [ ]:
model = SimpleRNN(len(vocab))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop

for epoch in range(epochs):

    total_loss = 0

    for question, answer in dataloader:

        optimizer.zero_grad()

        # forward pass

        output = model(question)

        # loss -> output shape should match (1,324) , answer (1)
        loss = criterion(output, answer[0])

        # gradients
        loss.backward()

        # update grads
        optimizer.step()

        # update loss
        total_loss = total_loss + loss.item()

    print(f"Epoch: {epoch+1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 523.9428
Epoch: 2, Loss: 456.0293
Epoch: 3, Loss: 380.4749
Epoch: 4, Loss: 316.2221
Epoch: 5, Loss: 263.5630
Epoch: 6, Loss: 215.0769
Epoch: 7, Loss: 170.6146
Epoch: 8, Loss: 132.6200
Epoch: 9, Loss: 101.7078
Epoch: 10, Loss: 77.4056
Epoch: 11, Loss: 59.6265
Epoch: 12, Loss: 46.6905
Epoch: 13, Loss: 36.9386
Epoch: 14, Loss: 29.7879
Epoch: 15, Loss: 24.7625
Epoch: 16, Loss: 20.7210
Epoch: 17, Loss: 17.4844
Epoch: 18, Loss: 14.8213
Epoch: 19, Loss: 12.7754
Epoch: 20, Loss: 11.0139


![image.png](attachment:image.png)

![image.png](attachment:image.png)

In [ ]:
# predictions
def predict(model, question, threshold=0.5):

    # convert question to numbers
    numerical_question = text_to_indices(question, vocab)

    # tensor
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)  # add 1 dimsenion at first

    # send to model
    output = model(question_tensor)

    # convert logits  to probabilities
    probs = torch.nn.functional.softmax(output, dim=1)

    # find index of max probs
    value, index = torch.max(probs, dim=1)

    if value < threshold:
        print("I don't know")
    else:
        print(list(vocab.keys())[index])

In [ ]:
predict(model, "what is it")

I don't know


In [ ]:
predict(model, "what is capital of france")

paris
